In [1]:
using Random, Distributions, Statistics, Plots
gr() 

Plots.GRBackend()

In [2]:
#maqモデル
function extract_y_t(y_t_array,k)
    if k <= length(y_t_array)
        return y_t_array[(length(y_t_array)+1-k)]
    else
        return 0
    end
end

function arma_pq(y_0, c, sigma, phi_array, theta_array, y_length)
    prepend!([1.0], theta_array)
    y_t_array = []
    noise_array = rand(Normal(0,sigma) , y_length)
    for i in 1:y_length
        if i == 0
            append!(y_t_array,y_0)
        else
            y_t = 0
            if i < length(theta_array)
                t = i
            else
                t =  length(theta_array)
            end
            for j in 1:length(phi_array)
                y_t += phi_array[j]*extract_y_t(y_t_array,j)
            end
            append!(y_t_array,c+y_t+sum([noise_array[i-(k-1)]*theta_array[k] for k in 1:t]))
        end
    end
    return y_t_array
end

arma_pq (generic function with 1 method)

In [ ]:
#最尤法で推定
#対数尤度を返す
function Likelihood_for_AR1(ar_proc)
    mean_y_t = mean(ar_proc[2:length(ar_proc)])
    mean_y_t_1 = mean(ar_proc[1:(length(ar_proc)-1)])
    phi = sum([(ar_proc[i]-mean_y_t)*(ar_proc[i-1]-mean_y_t_1)  for i in 2:(length(ar_proc))])/sum([(ar_proc[i-1]-mean_y_t_1)^2  for i in 2:(length(ar_proc))])
    c = mean_y_t - phi*mean_y_t_1
    s = sum([(ar_proc[i]-c-phi*ar_proc[i-1])^2  for i in 2:(length(ar_proc))])/(length(ar_proc)-1)
    return phi,c,s
end